In [1]:
!pip install fredapi

from fredapi import Fred

API_KEY = "2bef5770d514f4bcd9c45a8686583e74"
fred = Fred(api_key=API_KEY)

In [2]:
from fredapi import Fred

API_KEY = "2bef5770d514f4bcd9c45a8686583e74"
fred = Fred(api_key=API_KEY)

In [3]:
from fredapi import Fred
import pandas as pd
import numpy as np

# Initialize FRED API
API_KEY = "2bef5770d514f4bcd9c45a8686583e74"
fred = Fred(api_key=API_KEY)

# Fetch weekly gas prices (GASREGW)
gas_prices = fred.get_series("GASREGW", observation_start="2024-01-01")

# Fetch monthly CPI data (CPIAUCSL)
cpi_data = fred.get_series("CPIAUCSL", observation_start="2024-01-01")

# Convert CPI to weekly by forward-filling
cpi_weekly = cpi_data.resample("W").ffill()

# Convert gas prices to a DataFrame
gas_prices_df = pd.DataFrame(gas_prices, columns=["Gas Price ($)"])
gas_prices_df.index.name = "Week Start Date"
gas_prices_df.reset_index(inplace=True)
gas_prices_df["Week Start Date"] = gas_prices_df["Week Start Date"].astype(str)

# Convert CPI data to a DataFrame
cpi_df = pd.DataFrame(cpi_weekly, columns=["CPI"])
cpi_df.index.name = "Week Start Date"
cpi_df.reset_index(inplace=True)
cpi_df["Week Start Date"] = cpi_df["Week Start Date"].astype(str)

# Load furniture sales dataset
sales_df = pd.read_csv("/content/weekly_furniture_sales_data.csv")

# Merge sales data with FRED data
merged_df = sales_df.merge(gas_prices_df, on="Week Start Date", how="left")
merged_df = merged_df.merge(cpi_df, on="Week Start Date", how="left")

# Save the final dataset
merged_df.to_csv("weekly_furniture_sales_with_economic_data.csv", index=False)

print("Merged dataset saved as 'weekly_furniture_sales_with_economic_data.csv'")


Merged dataset saved as 'weekly_furniture_sales_with_economic_data.csv'


In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from fredapi import Fred
import os
from email.mime.text import MIMEText

# Initialize FRED API
API_KEY = "2bef5770d514f4bcd9c45a8686583e74"
fred = Fred(api_key=API_KEY)

# Define function to generate simulated sales data
def generate_sales_data(start_date, weeks=4):
    """Generate synthetic sales data for the given weeks."""
    product_ids = range(1, 51)
    furniture_products = [
        "Sofa", "Dining Table", "Chair", "Bed Frame", "Nightstand", "Bookshelf", "TV Stand", "Coffee Table",
        "Recliner", "Dresser", "Wardrobe", "Office Desk", "Computer Chair", "Futon", "End Table", "Bar Stool",
        "Patio Set", "Sideboard", "Loveseat", "Chaise Lounge", "Sectional Sofa", "File Cabinet", "Console Table",
        "Bench", "Hutch", "Buffet", "Storage Ottoman", "Accent Chair", "Bunk Bed", "Mattress", "Headboard",
        "Room Divider", "Floating Shelves", "Kitchen Island", "Rolling Cart", "Foldable Table", "Standing Desk",
        "Ladder Shelf", "Bookcase", "Vanity Table", "Makeup Chair", "Chest of Drawers", "Entryway Table",
        "Hall Tree", "Gaming Chair", "Bean Bag", "Murphy Bed", "Plant Stand", "Shoe Rack", "Coat Rack"
    ]

    sales_data = []
    for i in range(weeks):
        week_start = start_date + timedelta(weeks=i)
        for product_id, product_name in zip(product_ids, furniture_products):
            price = np.round(np.random.uniform(50, 2000), 2)
            discount = np.random.choice([0, np.round(np.random.uniform(5, 30), 2)], p=[0.7, 0.3])
            units_sold = np.random.randint(5, 50)
            revenue = units_sold * price * (1 - discount / 100)

            sales_data.append([
                week_start.strftime("%Y-%m-%d"), product_id, product_name, units_sold, price, discount, round(revenue, 2), "USA"
            ])

    return pd.DataFrame(sales_data, columns=["Week Start Date", "Product ID", "Product Name", "Units Sold", "Price ($)", "Discount Percentage", "Revenue ($)", "Region"])

# Fetch FRED data
def fetch_fred_data():
    """Fetch latest economic data from FRED."""
    gas_prices = fred.get_series("GASREGW", observation_start="2024-01-01")
    cpi_data = fred.get_series("CPIAUCSL", observation_start="2024-01-01")

    gas_prices_df = pd.DataFrame(gas_prices, columns=["Gas Price ($)"]).reset_index()
    gas_prices_df.columns = ["Week Start Date", "Gas Price ($)"]
    gas_prices_df["Week Start Date"] = gas_prices_df["Week Start Date"].astype(str)

    cpi_weekly = cpi_data.resample("W").ffill().reset_index()
    cpi_weekly.columns = ["Week Start Date", "CPI"]
    cpi_weekly["Week Start Date"] = cpi_weekly["Week Start Date"].astype(str)

    return gas_prices_df, cpi_weekly

# Main automation function
def update_data():
    """Automate sales and economic data updates."""
    today = datetime.today()
    start_date = today - timedelta(days=today.weekday())  # Start from the last Monday

    # Generate new sales data
    new_sales_data = generate_sales_data(start_date)

    # Fetch latest FRED data
    gas_prices_df, cpi_df = fetch_fred_data()

    # Merge datasets
    merged_df = new_sales_data.merge(gas_prices_df, on="Week Start Date", how="left")
    merged_df = merged_df.merge(cpi_df, on="Week Start Date", how="left")

    # Save dataset
    file_path = "weekly_furniture_sales_with_economic_data.csv"
    if os.path.exists(file_path):
        merged_df.to_csv(file_path, mode='a', header=False, index=False)  # Append data
    else:
        merged_df.to_csv(file_path, index=False)  # Create new file

    print(f"Data updated and saved to {file_path}")

# Run the update
update_data()


Data updated and saved to weekly_furniture_sales_with_economic_data.csv


In [7]:
import smtplib
import socket
import logging
from email.mime.text import MIMEText
socket.gethostbyname("smtp.gmail.com")

# Data integrity checks
def check_data_integrity(df):
    """Check for missing values, anomalies, and alignment issues."""
    issues = []

    # Check for missing values
    if df.isnull().sum().sum() > 0:
        issues.append("Missing values detected in the dataset.")

    # Detect anomalies in sales trends
    if (df['Units Sold'] > df['Units Sold'].mean() * 3).any():
        issues.append("Unusual spikes detected in sales data.")
    if (df['Units Sold'] < df['Units Sold'].mean() / 3).any():
        issues.append("Unusual drops detected in sales data.")

    # Check for missing product entries
    missing_products = set(range(1, 51)) - set(df['Product ID'].unique())
    if missing_products:
        issues.append(f"Missing products detected: {missing_products}")

    return issues

# Alert system
# Your credentials (Replace with your actual details)
EMAIL_ADDRESS = "narendrav64@gmail.com"
EMAIL_PASSWORD = "inalekvzhgcytmqs"  # Use the generated App Password

# Email message
msg = MIMEText("Test email body")
msg['Subject'] = "Test Subject"
msg['From'] = EMAIL_ADDRESS
msg['To'] = "narendrav64@gmail.com"

try:
    # Connect to Gmail SMTP server with TLS
    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()  # Upgrade connection to secure encrypted SSL/TLS
    server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)  # Authenticate
    server.sendmail(msg['From'], [msg['To']], msg.as_string())  # Send email
    server.quit()
    print("✅ Email sent successfully!")
except smtplib.SMTPAuthenticationError:
    print("❌ Authentication failed! Check email/password settings.")
except Exception as e:
    print(f"❌ An error occurred: {e}")


# Main automation function
def update_data():
    """Automate sales and economic data updates."""
    today = datetime.today()
    start_date = today - timedelta(days=today.weekday())  # Start from the last Monday

    # Generate new sales data
    new_sales_data = generate_sales_data(start_date)

    # Fetch latest FRED data
    gas_prices_df, cpi_df = fetch_fred_data()

    # Merge datasets
    merged_df = new_sales_data.merge(gas_prices_df, on="Week Start Date", how="left")
    merged_df = merged_df.merge(cpi_df, on="Week Start Date", how="left")

    # Perform data integrity checks
    issues = check_data_integrity(merged_df)
    def send_alert(issues):
     if issues:  # This line is correctly indented
        print("Data integrity issues detected:", issues)  # Proper indentation inside if-statement
     else:
        print("No issues found.")  # Another indented block


    # Save dataset
    file_path = "weekly_furniture_sales_with_economic_data.csv"
    if os.path.exists(file_path):
        merged_df.to_csv(file_path, mode='a', header=False, index=False)  # Append data
    else:
        merged_df.to_csv(file_path, index=False)  # Create new file

    print(f"Data updated and saved to {file_path}")

# Run the update
update_data()


✅ Email sent successfully!
Data updated and saved to weekly_furniture_sales_with_economic_data.csv
